In [ ]:
from stmpy import Driver, Machine

class Buzzer: 
    def on_init(self):
        print("INIT")


    def send...


t0 = {"source": "initial", "target": "idle", "effect": "on_init"}

t1 = {
    "trigger": "message",
    "source": "idle",
    "target": "waiting_for_buzz", 
    "effect": "start_timer('t', 20000)", 
}

t2 = {
    "trigger": "t",
    "source": "waiting_for_buzz",
    "target": "idle",  
}

t3 = {
    "trigger": "buzz",
    "source": "waiting_for_buzz",
    "target": "participant_answer", 
    "effect": "stop_timer('t')", 
}

t4 = {
    "trigger": "reset",
    "source": "participant_answer",
    "target": "idle", 
}


In [ ]:
from threading import Thread
import paho.mqtt.client as mqtt

class MQTT_Client: 
    def __init__(self):
        self.client = mqtt.Client()
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

    def on_connect(self, client, userdata, flags, rc): 
        print("on_connect(): {}".format(mqtt.connack_string(rc)))

    def on_message(self, client, userdata, msg):
        print("on_message(): topic: {}".format(msg.topic))
        self.stm_driver.send("message", "quiz_buzzer")
    
    def start(self, broker, port):
        print("Connecting to {}:{}".format(broker, port))
        self.client.connect(broker, port)

        self.client.subscribe("")

        try:
            thread = Thread(target=self.client.loop_forever)
            thread.start()
        except KeyboardInterrupt:
            print("Interrupted")
            self.client.disconnect()

In [1]:
broker, port = "mqtt.item.ntnu.no", 1883
buzzer = Buzzer()

buzzer_machine = Machine(transitions=[t0, t1, t2, t3, t4], obj=buzzer, name="quiz_buzzer")
buzzer.stm = buzzer_machine

driver = Driver()
driver.add_machine(buzzer_machine)

myclient = MQTT_Client()
buzzer.mqtt_client = myclient.client
myclient.stm_driver = driver

driver.start()
myclient.start(broker, port)


NameError: name 'Buzzer' is not defined